# Recommender

In [ ]:
import json

import requests
import torch
from transformers import DistilBertModel, DistilBertTokenizer

In [ ]:
model_class, tokenizer_class, pretrained_weights = DistilBertModel, DistilBertTokenizer, 'distilbert-base-cased'

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
def embed(sentences):
    # Not nice but works:
    # encode twice to figure out max. encoded length of the input sentences
    encoded = [tokenizer.encode(s, add_special_tokens=True) for s in sentences]
    max_encoded_len = max((len(e) for e in encoded))
    encoded = [tokenizer.encode(s, add_special_tokens=True, max_length=max_encoded_len, pad_to_max_length=True) for s in sentences]

    input_ids = torch.tensor(encoded)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]  # Models outputs are now tuples
    n = torch.mean(last_hidden_states, 1).numpy()
    l = [list(n[i]) for i in range(n.shape[0])]
    return l

In [ ]:
with requests.get("http://sql_app:80/users/1") as r:
    j = json.loads(r.content)

In [ ]:
articles_rated_pos = []
for rating in j["ratings"]:
    value = rating["value"]
    article_id = rating["article_id"]
    if value > 0:
        articles_rated_pos.append(article_id)

In [ ]:
articles_rated_pos

In [ ]:
# Get articles